# Predict reading scores using AFQ-Insight

In [1]:
import afqinsight as afqi
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
import palettable
import pandas as pd
import pickle

from mpl_toolkits.mplot3d import Axes3D

from bokeh.io import output_notebook
from bokeh.embed import file_html
from bokeh.layouts import row, column, widgetbox
from bokeh.models import BoxAnnotation, BoxSelectTool, ColorBar, CustomJS, HoverTool, Label, Legend, Range1d, Title, Whisker
from bokeh.models.tickers import FixedTicker
from bokeh.models.annotations import LegendItem
from bokeh.palettes import Spectral10, Cividis256, Category10_10, Category10
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.resources import CDN
from bokeh.models.mappers import LinearColorMapper

from sklearn.decomposition import PCA

%matplotlib notebook

In [2]:
output_notebook()

Loading BokehJS ...

## Load the data

In [3]:
cols_unadj = [
    'ReadEng_Unadj',
    'PicVocab_Unadj',
    'PicSeq_Unadj',
    'CardSort_Unadj',
    'Flanker_Unadj',
    'ProcSpeed_Unadj'
]

cols_ageadj = [
    'ReadEng_AgeAdj',
    'PicVocab_AgeAdj',
    'PicSeq_AgeAdj',
    'CardSort_AgeAdj',
    'Flanker_AgeAdj',
    'ProcSpeed_AgeAdj'
]

In [8]:
afq_data = afqi.load_afq_data(
    '../data/raw/reading_data',
    target_cols=cols_unadj
)

x, y, groups, columns, bias_index = (
    afq_data.x,
    afq_data.y,
    afq_data.groups,
    afq_data.columns,
    afq_data.bias_index
)

We have one subject with some missing scores.

In [9]:
y.loc[y.isna().any(axis='columns'), :]

,ReadEng_Unadj,PicVocab_Unadj,PicSeq_Unadj,CardSort_Unadj,Flanker_Unadj,ProcSpeed_Unadj
subjectID,,,,,,
140319,118.0674,119.2123,118.39,NaN,115.57,133.26


There are plenty of strategies we could use for imputing their data. But since it's just one subject, let's just drop them from our dataset.

In [10]:
subject_mask = y.index != 140319
x = x[subject_mask, :]
y = y.dropna(axis='rows')

print(x.shape)
print(y.shape)

(399, 4001)
(399, 6)


In [21]:
df_x = pd.DataFrame(np.delete(x, bias_index, axis=1), columns=columns)

In [25]:
df_x.to_csv('/Users/richford/Desktop/reading_score_features.csv')

In [ ]:
pca = PCA(n_components=1)
y_pca = pca.fit_transform(y)
print(pca.explained_variance_ratio_)

In [ ]:
y.columns

In [ ]:
y.loc[:, 'pca_ageadj'] = y_pca

In [ ]:
y.index = y.index.map(str)

In [ ]:
y_pca = y['pca_ageadj']
y_pca.index[0]

## Find the optimal feature coefficients $\widehat{\beta}$

We search for the optimal coefficients using square loss.

In [ ]:
hp_cv_res_square = afqi.fit_hyperparams_cv(
    x, y_pca, groups, bias_index=bias_index,
    max_evals_per_cv=2250, loss_type='square',
    score='rmse',
    trials_pickle_dir='./trials_reading_score_regression/cv10_rs42_square_rmse',
    verbose=1, random_state=42, clf_threshold=0.5
)

In [ ]:
# hp_cv_res_square = afqi.fit_hyperparams_cv(
#     x, y_pca, groups, bias_index=bias_index,
#     max_evals_per_cv=500, loss_type='square',
#     score='medae',
#     trials_pickle_dir='./trials_reading_score_regression/cv10_rs42_square_medae',
#     verbose=1, random_state=42, clf_threshold=0.5
# )

In [ ]:
[(r.alpha1, r.alpha2) for r in hp_cv_res_square]

In [ ]:
test_set_y_hat = pd.concat([
    pd.Series(data=cv.test.x.dot(cv.beta_hat),
              index=cv.test.y.index,
              name='yhat')
    for cv in hp_cv_res_square
])

df_y = pd.concat([y, test_set_y_hat], axis='columns', sort=True)
df_y['index'] = np.arange(len(y), dtype=np.int32)
df_y['stdres'] = (df_y['pca_ageadj'] - df_y['yhat']) / (np.std(df_y['pca_ageadj'] - df_y['yhat']))
df_y['subject_id'] = df_y.index
df_y.head()

In [ ]:
def print_results_summary(hp_cv_results):
    template = '{stat:15s} {mean:7.5g} ({var:7.5g})'
    test = [r.test for r in hp_cv_results]
    train = [r.train for r in hp_cv_results]
    test_rmse = [t.rmse for t in test]
    test_r2 = [t.r2 for t in test]
    train_rmse = [t.rmse for t in train]
    train_r2 = [t.r2 for t in train]

    print('Statistic         mean   (variance)')
    print('--------------  ------- ------------')
    print(template.format(stat='test RMSE', mean=np.mean(test_rmse), var=np.var(test_rmse)))
    print(template.format(stat='test R2', mean=np.mean(test_r2), var=np.var(test_r2)))
    print(template.format(stat='train RMSE', mean=np.mean(train_rmse), var=np.var(train_rmse)))
    print(template.format(stat='train R2', mean=np.mean(train_r2), var=np.var(train_r2)))

In [ ]:
[r.test.rmse for r in hp_cv_res_square]

In [ ]:
print_results_summary(hp_cv_res_square)

In [ ]:
p = figure(plot_width=600, plot_height=600, toolbar_location='above')
p.title.text = 'Residuals by subject'

source = ColumnDataSource(data=df_y)

hover = HoverTool(
    tooltips=[("Subject", "@subject_id"),
              ("Reading Score PC1", "@pca_ageadj")],
)
hover.point_policy = 'snap_to_data'
hover.line_policy = 'nearest'

p.circle(source=source,
         x='index',
         y='stdres',
         radius=2,
         fill_color=color, line_color=None)

p.xaxis.axis_label = 'Subject'
p.yaxis.axis_label = 'Standardized Residuals'

p.add_tools(hover)

# html = file_html(p, CDN, 'my plot')
# with open(op.abspath('../docs/img/bokeh_plots/regression_residuals_by_subject.html'), 'w') as fp:
#     fp.write(html)
show(p)

In [ ]:
p = figure(plot_width=500, plot_height=500, toolbar_location='above')
p_res = figure(plot_width=500, plot_height=500, toolbar_location='above')
p.title.text = 'Predicted vs. Actual Reading Scores for test splits'
p_res.title.text = 'Residuals for test splits'

source = ColumnDataSource(data=df_y)

hover = HoverTool(
    tooltips=[("Subject", "@subject_id"),
              ("Reading Score PC", "@pca_ageadj"),
              ("Residual", "@stdres")],
)
hover.point_policy = 'snap_to_data'
hover.line_policy = 'nearest'

p.scatter(source=source,
          x='yhat',
          y='pca_ageadj',
          radius=1,
          fill_color=color,
          line_color=None)

p_res.scatter(source=source,
              x='yhat',
              y='stdres',
              radius=1,
              fill_color=color,
              line_color=None)

p.xaxis.axis_label = 'Predicted Score'
p.yaxis.axis_label = 'Actual Score'

p_res.xaxis.axis_label = 'Predicted Score'
p_res.yaxis.axis_label = 'Standardized Residuals'

p.add_tools(hover)
p_res.add_tools(hover)

layout = row([p, p_res])

# html = file_html(layout, CDN, 'my plot')
# with open(op.abspath('../docs/img/bokeh_plots/regression_residuals.html'), 'w') as fp:
#     fp.write(html)
show(layout)

In [ ]:
p = figure(plot_width=700, plot_height=700, toolbar_location='above')
p.title.text = 'Predicted Reading Score for each CV split'
p.add_layout(
    Title(text='Click on legend entries to hide/show the corresponding lines',
          align="left"), 'right'
)

names = ['cv_idx = {i:d}'.format(i=i) for i in range(len(hp_cv_res_square))]

hover = HoverTool(
    tooltips=[("index", "$index"),],
    mode='vline',
)
hover.point_policy = 'snap_to_data'
hover.line_policy = 'nearest'

y_hat_mean = np.mean(np.array([x.dot(res.beta_hat) for res in hp_cv_res_square]), axis=0)
y_hat_std = np.std(np.array([x.dot(res.beta_hat) for res in hp_cv_res_square]), axis=0)

for res, color, name in zip(hp_cv_res_square, Spectral10, names):
    p.scatter(y_pca, x.dot(res.beta_hat),
              radius=0.3,
              fill_color=color, line_color=None,
              fill_alpha=0.75, legend=name)

p.scatter(y_pca, y_hat_mean, line_color=None, radius=0.5, legend='mean')
# p.add_layout(Whisker(base=y_pca,
#                      upper=y_hat_mean+y_hat_std,
#                      lower=y_hat_mean-y_hat_std,
#                      level='overlay'))

p.add_tools(hover)
p.legend.location = 'top_left'
p.legend.click_policy = 'hide'

show(p)

Using the hover tool on the chart above, we can see that subjects 05, 07, 16, 19, 30, 32, 35, 36 are all hard to classify (they are consistently closer to the classification threshold of 0.5). We should fire up the AFQ browser and look at how these subjects compare to the rest of the subjects in their group.

Here are links to a running instance of AFQ-Browser with the hard to classify subjects:
- [False negatives](https://yeatmanlab.github.io/Sarica_2017/?table[prevSort][count]=2&table[prevSort][order]=ascending&table[prevSort][key]=&table[sort][count]=2&table[sort][order]=ascending&table[sort][key]=class&table[selectedRows][subject_005]=true&table[selectedRows][subject_007]=true&table[selectedRows][subject_016]=true&table[selectedRows][subject_019]=true&table[selectedRows][subject_030]=false&table[selectedRows][subject_032]=false&table[selectedRows][subject_035]=false&table[selectedRows][subject_036]=false&plots[checkboxes][right-corticospinal]=true&plots[zoom][rd][scale]=1&plots[zoom][rd][translate][0]=-3&plots[zoom][rd][translate][1]=-21&plots[zoom][fa][scale]=2.1140360811227614&plots[zoom][fa][translate][0]=-27.244995845837778&plots[zoom][fa][translate][1]=-106.10468474511174&plots[plotKey]=fa&plots[errorType]=stderr&plots[lineOpacity]=0.09355440414507772)
- [False positives](https://yeatmanlab.github.io/Sarica_2017/?table[prevSort][count]=2&table[prevSort][order]=ascending&table[prevSort][key]=&table[sort][count]=2&table[sort][order]=ascending&table[sort][key]=class&table[selectedRows][subject_005]=false&table[selectedRows][subject_007]=false&table[selectedRows][subject_016]=false&table[selectedRows][subject_019]=false&table[selectedRows][subject_030]=true&table[selectedRows][subject_032]=true&table[selectedRows][subject_035]=true&table[selectedRows][subject_036]=true&plots[checkboxes][right-corticospinal]=true&plots[zoom][rd][scale]=1&plots[zoom][rd][translate][0]=-3&plots[zoom][rd][translate][1]=-21&plots[zoom][fa][scale]=2.1140360811227614&plots[zoom][fa][translate][0]=-27.244995845837778&plots[zoom][fa][translate][1]=-106.10468474511174&plots[plotKey]=fa&plots[errorType]=stderr&plots[lineOpacity]=0.09355440414507772)

# Feature Importance

Let's sort the features by their importance

In [ ]:
feature_dicts = afqi.multicol2dicts(columns, tract_symmetry=False)

mean_beta = np.mean(np.array(
    [np.delete(res.beta_hat, bias_index) for res in hp_cv_res_square]
), axis=0)

mean_beta_converged = np.mean(np.array(
    [np.delete(res.beta_hat, bias_index) for res in ([hp_cv_res_square[0]] + [hp_cv_res_square[3]] + [hp_cv_res_square[5]] + [hp_cv_res_square[8]])]
), axis=0)

sorted_features = afqi.sort_features(feature_dicts, mean_beta)

sorted_features[0:50]

It's nice to see the top few features in a sorted list, but let's plot the features to get a feel for their distributions

In [ ]:
beta_hats = afqi.beta_hat_by_groups(mean_beta, columns=columns, drop_zeros=True)
beta_hats_converged = afqi.beta_hat_by_groups(mean_beta_converged, columns=columns, drop_zeros=True)

In [ ]:
unfolded_beta = afqi.transform.unfold_beta_hat_by_metrics(
    beta_hat=mean_beta_converged,
    columns=columns
)

In [ ]:
afqi.plot.plot_unfolded_beta(
    unfolded_beta=unfolded_beta,
    output_html='../docs/img/bokeh_plots/reading_regression_unfolded_beta.html',
    width=1500,
    height=800
)

In [ ]:
y_pca_series = pd.Series(y_pca, index=y.index)

In [ ]:
afqi.plot.plot_pca_space(
    x=np.delete(x, bias_index, axis=1),
    y=y_pca_series,
    beta=mean_beta,
    target_name='Combined Score',
    plot_type='regression',
    output_html='../docs/img/bokeh_plots/reading_regression_pca_both.html',
    width=750,
    height=750
)

In [ ]:
afqi.plot.plot_pca_space(
    x=np.delete(x, bias_index, axis=1),
    y=y_pca_series,
    beta=mean_beta,
    target_name='Combined Score',
    plot_type='regression',
    plot_both=False,
    output_html='../docs/img/bokeh_plots/reading_regression_pca_sgl_only.html',
    width=750,
    height=750
)